In [413]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm # pip install statsmodels

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import linear_model
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline

import tensorflow as tf
from tensorflow.keras import metrics
from tensorflow.keras import backend as K

import keras
from keras import regularizers
from keras import layers

from keras.models import Sequential

from keras.callbacks import EarlyStopping

from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LayerNormalization, BatchNormalization

import os


***
# DO NOT REMOVE

In [414]:
# root_logdir = os.path.join(os.curdir, "my_logs")

# def get_run_logdir(config):
#     import time
#     run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
#     return os.path.join(root_logdir, f"{run_id}-{config['epochs']}-{config['batch_size']}-{config['lr']}-{config['patience']}")

***

In [415]:
df = pd.read_csv('./df_agg_57.csv')
print(df.shape)
df.head()

(57, 117)


,storm,deaths,min_time,max_time,avg_elevation,min_elevation,max_elevation,med_elevation,avg_temperature_2m,min_temperature_2m,...,med_soil_moisture_7_to_28cm,avg_soil_moisture_28_to_100cm,min_soil_moisture_28_to_100cm,max_soil_moisture_28_to_100cm,med_soil_moisture_28_to_100cm,avg_soil_moisture_100_to_255cm,min_soil_moisture_100_to_255cm,max_soil_moisture_100_to_255cm,med_soil_moisture_100_to_255cm,target_cost
0,Georges,16,1998-09-15,1998-10-01,63.60,0.0,1445.0,0.0,26.18,18.2,...,0.0,0.13,0.0,0.52,0.0,0.11,0.0,0.50,0.0,11191.0
1,Isidore,5,2002-09-14,2002-09-27,26.16,0.0,431.0,0.0,25.72,12.3,...,0.0,0.11,0.0,0.49,0.0,0.09,0.0,0.50,0.0,1943.5
2,Lili,2,2002-09-21,2002-10-04,3.77,0.0,77.0,0.0,26.40,21.6,...,0.0,0.06,0.0,0.51,0.0,0.05,0.0,0.44,0.0,1866.6
3,Ivan,57,2004-09-02,2004-09-24,35.21,-2.0,891.0,0.0,25.90,12.1,...,0.0,0.06,0.0,0.52,0.0,0.06,0.0,0.50,0.0,33005.8
4,Dorian,10,2019-08-24,2019-09-09,4.11,0.0,183.0,0.0,25.50,7.7,...,0.0,0.03,0.0,0.43,0.0,0.02,0.0,0.38,0.0,1915.9


In [416]:
all_original_columns = list(df.columns)
all_original_columns

['storm',
 'deaths',
 'min_time',
 'max_time',
 'avg_elevation',
 'min_elevation',
 'max_elevation',
 'med_elevation',
 'avg_temperature_2m',
 'min_temperature_2m',
 'max_temperature_2m',
 'med_temperature_2m',
 'avg_relativehumidity_2m',
 'min_relativehumidity_2m',
 'max_relativehumidity_2m',
 'med_relativehumidity_2m',
 'avg_dewpoint_2m',
 'min_dewpoint_2m',
 'max_dewpoint_2m',
 'med_dewpoint_2m',
 'avg_apparent_temperature',
 'min_apparent_temperature',
 'max_apparent_temperature',
 'med_apparent_temperature',
 'avg_precipitation',
 'min_precipitation',
 'max_precipitation',
 'med_precipitation',
 'avg_rain',
 'min_rain',
 'max_rain',
 'med_rain',
 'avg_pressure_msl',
 'min_pressure_msl',
 'max_pressure_msl',
 'med_pressure_msl',
 'avg_surface_pressure',
 'min_surface_pressure',
 'max_surface_pressure',
 'med_surface_pressure',
 'avg_cloudcover',
 'min_cloudcover',
 'max_cloudcover',
 'med_cloudcover',
 'avg_cloudcover_low',
 'min_cloudcover_low',
 'max_cloudcover_low',
 'med_cloudc

In [417]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Columns: 117 entries, storm to target_cost
dtypes: float64(99), int64(15), object(3)
memory usage: 52.2+ KB


In [418]:
num_df = df.copy()

In [419]:
num_df.drop(columns=['storm'], axis=1, inplace=True)

In [420]:
num_df.head()

,deaths,min_time,max_time,avg_elevation,min_elevation,max_elevation,med_elevation,avg_temperature_2m,min_temperature_2m,max_temperature_2m,...,med_soil_moisture_7_to_28cm,avg_soil_moisture_28_to_100cm,min_soil_moisture_28_to_100cm,max_soil_moisture_28_to_100cm,med_soil_moisture_28_to_100cm,avg_soil_moisture_100_to_255cm,min_soil_moisture_100_to_255cm,max_soil_moisture_100_to_255cm,med_soil_moisture_100_to_255cm,target_cost
0,16,1998-09-15,1998-10-01,63.60,0.0,1445.0,0.0,26.18,18.2,29.4,...,0.0,0.13,0.0,0.52,0.0,0.11,0.0,0.50,0.0,11191.0
1,5,2002-09-14,2002-09-27,26.16,0.0,431.0,0.0,25.72,12.3,29.7,...,0.0,0.11,0.0,0.49,0.0,0.09,0.0,0.50,0.0,1943.5
2,2,2002-09-21,2002-10-04,3.77,0.0,77.0,0.0,26.40,21.6,31.8,...,0.0,0.06,0.0,0.51,0.0,0.05,0.0,0.44,0.0,1866.6
3,57,2004-09-02,2004-09-24,35.21,-2.0,891.0,0.0,25.90,12.1,29.7,...,0.0,0.06,0.0,0.52,0.0,0.06,0.0,0.50,0.0,33005.8
4,10,2019-08-24,2019-09-09,4.11,0.0,183.0,0.0,25.50,7.7,30.3,...,0.0,0.03,0.0,0.43,0.0,0.02,0.0,0.38,0.0,1915.9


In [421]:
# Convert the datetime column to a timestamp format (seconds since epoch)
num_df['min_time'] = pd.to_datetime(num_df['min_time'])
num_df['max_time'] = pd.to_datetime(num_df['max_time'])
num_df['min_time'] = num_df['min_time'].view('int64') // 10**9
num_df['max_time'] = num_df['max_time'].view('int64') // 10**9
num_df.head()

,deaths,min_time,max_time,avg_elevation,min_elevation,max_elevation,med_elevation,avg_temperature_2m,min_temperature_2m,max_temperature_2m,...,med_soil_moisture_7_to_28cm,avg_soil_moisture_28_to_100cm,min_soil_moisture_28_to_100cm,max_soil_moisture_28_to_100cm,med_soil_moisture_28_to_100cm,avg_soil_moisture_100_to_255cm,min_soil_moisture_100_to_255cm,max_soil_moisture_100_to_255cm,med_soil_moisture_100_to_255cm,target_cost
0,16,905817600,907200000,63.60,0.0,1445.0,0.0,26.18,18.2,29.4,...,0.0,0.13,0.0,0.52,0.0,0.11,0.0,0.50,0.0,11191.0
1,5,1031961600,1033084800,26.16,0.0,431.0,0.0,25.72,12.3,29.7,...,0.0,0.11,0.0,0.49,0.0,0.09,0.0,0.50,0.0,1943.5
2,2,1032566400,1033689600,3.77,0.0,77.0,0.0,26.40,21.6,31.8,...,0.0,0.06,0.0,0.51,0.0,0.05,0.0,0.44,0.0,1866.6
3,57,1094083200,1095984000,35.21,-2.0,891.0,0.0,25.90,12.1,29.7,...,0.0,0.06,0.0,0.52,0.0,0.06,0.0,0.50,0.0,33005.8
4,10,1566604800,1567987200,4.11,0.0,183.0,0.0,25.50,7.7,30.3,...,0.0,0.03,0.0,0.43,0.0,0.02,0.0,0.38,0.0,1915.9


In [422]:
constant_columns = num_df.columns[num_df.nunique() <= 1]
constant_columns


Index(['min_precipitation', 'min_rain', 'max_cloudcover', 'min_cloudcover_low',
       'max_cloudcover_low', 'max_cloudcover_mid', 'max_cloudcover_high',
       'min_et0_fao_evapotranspiration', 'min_soil_moisture_0_to_7cm',
       'min_soil_moisture_7_to_28cm', 'min_soil_moisture_28_to_100cm',
       'min_soil_moisture_100_to_255cm'],
      dtype='object')

In [423]:
list(num_df.dtypes)

[dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),


In [424]:
"""These columns don't provide any variability, and thus can't be correlated with other columns. 
When you try to compute the correlation of a constant column with any other column, the result is NaN (not a number) 
because the standard deviation of a constant column is zero, and you can't divide by zero in the correlation formula."""

constant_columns = num_df.columns[num_df.nunique() == 1]
constant_columns

Index(['min_precipitation', 'min_rain', 'max_cloudcover', 'min_cloudcover_low',
       'max_cloudcover_low', 'max_cloudcover_mid', 'max_cloudcover_high',
       'min_et0_fao_evapotranspiration', 'min_soil_moisture_0_to_7cm',
       'min_soil_moisture_7_to_28cm', 'min_soil_moisture_28_to_100cm',
       'min_soil_moisture_100_to_255cm'],
      dtype='object')

In [425]:
# Dropping constant columns
num_df = num_df.drop(columns=constant_columns)
print(num_df.shape)
num_df.head()

(57, 104)


,deaths,min_time,max_time,avg_elevation,min_elevation,max_elevation,med_elevation,avg_temperature_2m,min_temperature_2m,max_temperature_2m,...,avg_soil_moisture_7_to_28cm,max_soil_moisture_7_to_28cm,med_soil_moisture_7_to_28cm,avg_soil_moisture_28_to_100cm,max_soil_moisture_28_to_100cm,med_soil_moisture_28_to_100cm,avg_soil_moisture_100_to_255cm,max_soil_moisture_100_to_255cm,med_soil_moisture_100_to_255cm,target_cost
0,16,905817600,907200000,63.60,0.0,1445.0,0.0,26.18,18.2,29.4,...,0.15,0.52,0.0,0.13,0.52,0.0,0.11,0.50,0.0,11191.0
1,5,1031961600,1033084800,26.16,0.0,431.0,0.0,25.72,12.3,29.7,...,0.14,0.52,0.0,0.11,0.49,0.0,0.09,0.50,0.0,1943.5
2,2,1032566400,1033689600,3.77,0.0,77.0,0.0,26.40,21.6,31.8,...,0.07,0.52,0.0,0.06,0.51,0.0,0.05,0.44,0.0,1866.6
3,57,1094083200,1095984000,35.21,-2.0,891.0,0.0,25.90,12.1,29.7,...,0.06,0.52,0.0,0.06,0.52,0.0,0.06,0.50,0.0,33005.8
4,10,1566604800,1567987200,4.11,0.0,183.0,0.0,25.50,7.7,30.3,...,0.04,0.44,0.0,0.03,0.43,0.0,0.02,0.38,0.0,1915.9


# Feature Correlations

In [426]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColorBar, LinearColorMapper, ColumnDataSource, HoverTool
from bokeh.transform import transform
from bokeh.layouts import column
from bokeh.models.tickers import BasicTicker

output_notebook()

correlation_data = num_df.corr()

correlation_matrix = pd.DataFrame({
    'x_name': correlation_data.columns.repeat(len(correlation_data.index)),
    'y_name': np.tile(correlation_data.index, len(correlation_data.columns)),
    'values': correlation_data.values.ravel()
})

# Filter correlation_matrix for abs(values) >= 0.8
correlation_matrix = correlation_matrix[correlation_matrix['values'].abs() >= 0.8]

source = ColumnDataSource(correlation_matrix)

colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "darkred"]
mapper = LinearColorMapper(palette=colors, low=-1, high=1)  # As correlations range between -1 and 1

unique_factors = list(correlation_data.columns)
p = figure(title="Heatmap",
           x_range=unique_factors, y_range=unique_factors, tools="", width=1500, height=1500)

p.xaxis.major_label_orientation = "vertical"

p.rect(x="x_name", y="y_name", width=1, height=1, source=source,
       line_color='white', fill_color=transform('values', mapper))

color_bar = ColorBar(color_mapper=mapper, location=(0, 0),
                     ticker=BasicTicker(desired_num_ticks=len(colors)))

p.add_layout(color_bar, 'right')

# Adding hover tool for interactivity
hover = HoverTool()
hover.tooltips = [("X", "@x_name"), ("Y", "@y_name"), ("Correlation", "@values")]
p.add_tools(hover)

show(p)


Loading BokehJS ...

In [427]:
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LinearRegression
# import pandas as pd

# # Assume df is your DataFrame
# X = num_df.iloc[:, :-1]  # Predictor columns
# y = num_df['target_cost']  # Target column

# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# # Fit model
# model = LinearRegression()
# model.fit(X_scaled, y)

# # Get feature importances (coefficients in this case)
# feature_importance = model.coef_

# # Pair feature names with importance
# features_list = list(X.columns)
# feature_importance_list = list(zip(features_list, feature_importance))

# # Sort by absolute value of coefficient
# sorted_features = sorted(feature_importance_list, key=lambda x: abs(x[1]), reverse=True)

# # Select top 50 features
# top_50_features = [feature for feature, _ in sorted_features[:50]]

# print("Top 50 most influential features:", top_50_features)



In [428]:
# import statsmodels.api as sm
# import matplotlib.pyplot as plt
# import seaborn as sns
# from scipy import stats
# # Assume X and y are your predictors and target variable
# X = num_df.iloc[:, :-1]  # Predictor columns
# y = num_df['target_cost']  # Target column

# # Fit OLS model
# X_with_const = sm.add_constant(X_scaled)  # Adds constant term to predictors
# ols_model = sm.OLS(y, X_with_const).fit()

# # Residuals
# residuals = ols_model.resid

# # Fitted values
# fitted = ols_model.fittedvalues

# # Residual vs Fitted plot for Linearity and Homoscedasticity
# plt.scatter(fitted, residuals)
# plt.axhline(0, color='r', linestyle='--')
# plt.xlabel('Fitted values')
# plt.ylabel('Residuals')
# plt.show()

# # QQ plot for Normality
# sm.qqplot(residuals, line='s')
# plt.show()

# # VIF for Multicollinearity
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# vif_data = pd.DataFrame()
# vif_data["feature"] = X.columns
# vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
# print(vif_data)


In [429]:
# top_50_features

In [430]:
# len(top_predictors)

In [431]:
correlation_matrix[correlation_matrix['values'].abs() >= 0.8]

,x_name,y_name,values
0,deaths,deaths,1.000000
105,min_time,min_time,1.000000
106,min_time,max_time,1.000000
209,max_time,min_time,1.000000
210,max_time,max_time,1.000000
...,...,...,...
10704,med_soil_moisture_100_to_255cm,med_soil_moisture_7_to_28cm,0.964593
10707,med_soil_moisture_100_to_255cm,med_soil_moisture_28_to_100cm,0.979436
10708,med_soil_moisture_100_to_255cm,avg_soil_moisture_100_to_255cm,0.834654
10710,med_soil_moisture_100_to_255cm,med_soil_moisture_100_to_255cm,1.000000


In [432]:
correlation_matrix

,x_name,y_name,values
0,deaths,deaths,1.000000
105,min_time,min_time,1.000000
106,min_time,max_time,1.000000
209,max_time,min_time,1.000000
210,max_time,max_time,1.000000
...,...,...,...
10704,med_soil_moisture_100_to_255cm,med_soil_moisture_7_to_28cm,0.964593
10707,med_soil_moisture_100_to_255cm,med_soil_moisture_28_to_100cm,0.979436
10708,med_soil_moisture_100_to_255cm,avg_soil_moisture_100_to_255cm,0.834654
10710,med_soil_moisture_100_to_255cm,med_soil_moisture_100_to_255cm,1.000000


In [433]:
prep_corr = correlation_matrix.copy()
prep_corr

,x_name,y_name,values
0,deaths,deaths,1.000000
105,min_time,min_time,1.000000
106,min_time,max_time,1.000000
209,max_time,min_time,1.000000
210,max_time,max_time,1.000000
...,...,...,...
10704,med_soil_moisture_100_to_255cm,med_soil_moisture_7_to_28cm,0.964593
10707,med_soil_moisture_100_to_255cm,med_soil_moisture_28_to_100cm,0.979436
10708,med_soil_moisture_100_to_255cm,avg_soil_moisture_100_to_255cm,0.834654
10710,med_soil_moisture_100_to_255cm,med_soil_moisture_100_to_255cm,1.000000


In [434]:
corrs_to_review = prep_corr[prep_corr['x_name'] != prep_corr['y_name']]
# corrs_to_review.to_csv('./corrs_to_review.csv', index=False)

In [435]:
reduced_df = num_df.copy()
# reduced_df.drop(columns=['avg_rain', 
#                         'med_rain', 
#                         'avg_dewpoint_2m', 
#                         'med_dewpoint_2m', 
#                         'min_dewpoint_2m', 
#                         'min_temperature_2m', 
#                         'med_temperature_2m',
#                         'avg_temperature_2m',
#                         'max_time',
#                         'avg_soil_moisture_0_to_7cm',
#                         'avg_soil_moisture_100_to_255cm',
#                         'med_vapor_pressure_deficit',
#                         'avg_soil_temperature_0_to_7cm',
#                         'avg_windspeed_10m',
#                         'max_soil_temperature_7_to_28cm',
#                         'min_soil_temperature_7_to_28cm',
#                         'avg_soil_temperature_7_to_28cm',
#                         'med_soil_temperature_7_to_28cm'
#                         ], 
#                         axis=1, inplace=True)

# reduced_df.drop(columns=[
#     'avg_rain',
#     'max_rain',
#     'med_rain',
#     'deaths',
    
#     ], 
#     axis=1, inplace=True)
reduced_df.shape

(57, 104)

In [436]:
# Find columns with only one unique value
single_value_columns = reduced_df.columns[reduced_df.nunique() == 1].tolist()
single_value_columns

[]

In [437]:
# Compute correlation matrix
correlation_matrix = reduced_df.corr()

# Find columns with absolute correlation > 0.8
columns_to_drop = set()
dropped = set()

for col in correlation_matrix.columns:
    if col not in dropped:
        # Exclude self-correlation
        correlated_cols = correlation_matrix.index[(correlation_matrix[col].abs() > 0.8) & (correlation_matrix.index != col)]
        for correlated_col in correlated_cols:
            columns_to_drop.add(correlated_col)
            dropped.add(correlated_col)

# Drop those columns
reduced_df.drop(columns=columns_to_drop, inplace=True)
reduced_df.drop(columns=['deaths'], inplace=True)

reduced_df.shape



(57, 45)

In [438]:
output_notebook()

correlation_data = reduced_df.corr()

correlation_matrix = pd.DataFrame({
    'x_name': correlation_data.columns.repeat(len(correlation_data.index)),
    'y_name': np.tile(correlation_data.index, len(correlation_data.columns)),
    'values': correlation_data.values.ravel()
})

# Filter correlation_matrix for abs(values) >= 0.8
# correlation_matrix = correlation_matrix[correlation_matrix['values'].abs() >= 0.8]

source = ColumnDataSource(correlation_matrix)

colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "darkred"]
mapper = LinearColorMapper(palette=colors, low=-1, high=1)  # As correlations range between -1 and 1

unique_factors = list(correlation_data.columns)
p = figure(title="Heatmap",
           x_range=unique_factors, y_range=unique_factors, tools="", width=1500, height=1500)

p.xaxis.major_label_orientation = "vertical"

p.rect(x="x_name", y="y_name", width=1, height=1, source=source,
       line_color='white', fill_color=transform('values', mapper))

color_bar = ColorBar(color_mapper=mapper, location=(0, 0),
                     ticker=BasicTicker(desired_num_ticks=len(colors)))

p.add_layout(color_bar, 'right')

# Adding hover tool for interactivity
hover = HoverTool()
hover.tooltips = [("X", "@x_name"), ("Y", "@y_name"), ("Correlation", "@values")]
p.add_tools(hover)

show(p)

Loading BokehJS ...

In [439]:
reduced_df.shape

(57, 45)

In [440]:
reduced_columns = list(reduced_df.columns)
reduced_columns

['min_time',
 'avg_elevation',
 'min_elevation',
 'max_elevation',
 'med_elevation',
 'avg_temperature_2m',
 'min_temperature_2m',
 'max_temperature_2m',
 'med_temperature_2m',
 'avg_relativehumidity_2m',
 'min_relativehumidity_2m',
 'max_relativehumidity_2m',
 'max_dewpoint_2m',
 'avg_apparent_temperature',
 'avg_precipitation',
 'max_precipitation',
 'avg_pressure_msl',
 'min_pressure_msl',
 'max_pressure_msl',
 'avg_surface_pressure',
 'avg_cloudcover',
 'min_cloudcover',
 'avg_cloudcover_low',
 'min_cloudcover_mid',
 'avg_cloudcover_high',
 'med_cloudcover_high',
 'avg_et0_fao_evapotranspiration',
 'max_et0_fao_evapotranspiration',
 'med_et0_fao_evapotranspiration',
 'avg_windspeed_10m',
 'min_windspeed_10m',
 'avg_winddirection_10m',
 'max_winddirection_10m',
 'max_winddirection_100m',
 'min_windgusts_10m',
 'max_soil_temperature_0_to_7cm',
 'max_soil_temperature_7_to_28cm',
 'avg_soil_temperature_100_to_255cm',
 'min_soil_temperature_100_to_255cm',
 'max_soil_temperature_100_to_2

In [441]:
cols_for_df = [
    'const', 
    'min_time',
    'avg_elevation',
    'min_elevation',
    'max_elevation',
    'med_elevation',
    'avg_temperature_2m',
    'min_temperature_2m',
    'max_temperature_2m',
    'med_temperature_2m',
    'avg_relativehumidity_2m',
    'min_relativehumidity_2m',
    'max_relativehumidity_2m',
    'max_dewpoint_2m',
    'avg_apparent_temperature',
    'avg_precipitation',
    'max_precipitation',
    'avg_pressure_msl',
    'min_pressure_msl',
    'max_pressure_msl',
    'avg_surface_pressure',
    'avg_cloudcover',
    'min_cloudcover',
    'avg_cloudcover_low',
    'min_cloudcover_mid',
    'avg_cloudcover_high',
    'med_cloudcover_high',
    'avg_et0_fao_evapotranspiration',
    'max_et0_fao_evapotranspiration',
    'med_et0_fao_evapotranspiration',
    'avg_windspeed_10m',
    'min_windspeed_10m',
    'avg_winddirection_10m',
    'max_winddirection_10m',
    'max_winddirection_100m',
    'min_windgusts_10m',
    'max_soil_temperature_0_to_7cm',
    'max_soil_temperature_7_to_28cm',
    'avg_soil_temperature_100_to_255cm',
    'min_soil_temperature_100_to_255cm',
    'max_soil_temperature_100_to_255cm',
    'avg_soil_moisture_0_to_7cm',
    'max_soil_moisture_0_to_7cm',
    'max_soil_moisture_28_to_100cm',
    'max_soil_moisture_100_to_255cm']

# multiple regression

In [442]:
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# X_optim_scaled = scaler.fit_transform(X_optim)
# X_optim_scaled = pd.DataFrame(X_optim_scaled, columns=X_optim.columns)

X = reduced_df.drop('target_cost', axis=1)  # Explanatory variables
y = reduced_df['target_cost']  # Dependent variable

# add constant to the model (it's a best practice)
X = sm.add_constant(X)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# split train/test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# regression base model
model_base = sm.OLS(y_train, X_train).fit()

print(model_base.summary())

                            OLS Regression Results                            
Dep. Variable:            target_cost   R-squared:                       0.974
Model:                            OLS   Adj. R-squared:                 -0.122
Method:                 Least Squares   F-statistic:                    0.8885
Date:                Tue, 31 Oct 2023   Prob (F-statistic):              0.705
Time:                        17:20:51   Log-Likelihood:                -456.74
No. Observations:                  45   AIC:                             1001.
Df Residuals:                       1   BIC:                             1081.
Df Model:                          43                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.813e-07   5.65e-07     -0.675      0.6

In [448]:
X_train_df = pd.DataFrame(X_train, columns=cols_for_df)
print(X_train_df.shape)
X_train_df.head()

(45, 45)


,const,min_time,avg_elevation,min_elevation,max_elevation,med_elevation,avg_temperature_2m,min_temperature_2m,max_temperature_2m,med_temperature_2m,...,min_windgusts_10m,max_soil_temperature_0_to_7cm,max_soil_temperature_7_to_28cm,avg_soil_temperature_100_to_255cm,min_soil_temperature_100_to_255cm,max_soil_temperature_100_to_255cm,avg_soil_moisture_0_to_7cm,max_soil_moisture_0_to_7cm,max_soil_moisture_28_to_100cm,max_soil_moisture_100_to_255cm
0,0.0,1.247750,-0.042786,0.226339,0.858352,-0.236802,-0.882261,-0.212131,-1.454231,-0.863126,...,-0.190840,-0.780923,-1.012978,0.090455,-0.057535,-1.188515,0.173229,-0.070324,-0.553308,0.322078
1,0.0,1.064653,0.834359,0.226339,1.433286,-0.236802,1.069728,0.856335,1.469935,0.822293,...,-0.071956,0.688243,1.327823,0.565476,0.605340,1.917973,0.443751,-0.070324,-0.255774,0.199689
2,0.0,-0.258497,0.045467,0.226339,-0.252842,-0.236802,0.157458,-0.853211,-0.797785,0.421003,...,-1.052750,-0.437075,-0.297733,0.016233,-0.912294,0.125769,-0.232554,-0.070324,0.636826,0.322078
3,0.0,0.901270,-0.135881,0.226339,0.200917,-0.236802,0.405649,0.571411,-0.618755,0.019713,...,0.254975,-0.499593,-0.622845,0.070663,0.099462,-0.471633,-0.638338,-0.070324,-0.702075,-0.045091
4,0.0,1.135807,-0.585037,0.226339,-0.946372,-0.236802,0.351986,-0.016246,1.469935,0.260487,...,-1.052750,0.188101,0.677600,-0.651764,-1.348396,-0.471633,-0.232554,-0.070324,-1.148375,0.077299


In [452]:
X_train_df.columns[1:]

Index(['min_time', 'avg_elevation', 'min_elevation', 'max_elevation',
       'med_elevation', 'avg_temperature_2m', 'min_temperature_2m',
       'max_temperature_2m', 'med_temperature_2m', 'avg_relativehumidity_2m',
       'min_relativehumidity_2m', 'max_relativehumidity_2m', 'max_dewpoint_2m',
       'avg_apparent_temperature', 'avg_precipitation', 'max_precipitation',
       'avg_pressure_msl', 'min_pressure_msl', 'max_pressure_msl',
       'avg_surface_pressure', 'avg_cloudcover', 'min_cloudcover',
       'avg_cloudcover_low', 'min_cloudcover_mid', 'avg_cloudcover_high',
       'med_cloudcover_high', 'avg_et0_fao_evapotranspiration',
       'max_et0_fao_evapotranspiration', 'med_et0_fao_evapotranspiration',
       'avg_windspeed_10m', 'min_windspeed_10m', 'avg_winddirection_10m',
       'max_winddirection_10m', 'max_winddirection_100m', 'min_windgusts_10m',
       'max_soil_temperature_0_to_7cm', 'max_soil_temperature_7_to_28cm',
       'avg_soil_temperature_100_to_255cm',
       'm

In [453]:
base_adj_rsq = model_base.rsquared_adj
base_adj_rsq

-0.12234964508588764

In [454]:
backward_elimination_adjr2s = {}
temp_adjr2 = 0
X_cols = X_train_df.columns[1:]

df_optimized = X_train_df.copy()

for col in list(X_cols):
    X_optimizer = df_optimized.copy()
    # print(X_optimizer.shape)
    X_optimizer.drop(columns=[col], axis=1, inplace=True)
    # X_optimizer = sm.add_constant(X_optimizer) #<< already have
    model_opt = sm.OLS(y_train, X_train).fit()
    curr_adjr2 = model_opt.rsquared_adj
    # print(curr_adjr2, base_adj_rsq)
    
    if curr_adjr2 >= base_adj_rsq:
        base_adj_rsq = curr_adjr2
    elif curr_adjr2 < base_adj_rsq:
        df_optimized.drop(columns=[col], axis=1, inplace=True)
     
        
    backward_elimination_adjr2s[col] = curr_adjr2

backward_elimination_adjr2s


{'min_time': -0.12234964508588764,
 'avg_elevation': -0.12234964508588764,
 'min_elevation': -0.12234964508588764,
 'max_elevation': -0.12234964508588764,
 'med_elevation': -0.12234964508588764,
 'avg_temperature_2m': -0.12234964508588764,
 'min_temperature_2m': -0.12234964508588764,
 'max_temperature_2m': -0.12234964508588764,
 'med_temperature_2m': -0.12234964508588764,
 'avg_relativehumidity_2m': -0.12234964508588764,
 'min_relativehumidity_2m': -0.12234964508588764,
 'max_relativehumidity_2m': -0.12234964508588764,
 'max_dewpoint_2m': -0.12234964508588764,
 'avg_apparent_temperature': -0.12234964508588764,
 'avg_precipitation': -0.12234964508588764,
 'max_precipitation': -0.12234964508588764,
 'avg_pressure_msl': -0.12234964508588764,
 'min_pressure_msl': -0.12234964508588764,
 'max_pressure_msl': -0.12234964508588764,
 'avg_surface_pressure': -0.12234964508588764,
 'avg_cloudcover': -0.12234964508588764,
 'min_cloudcover': -0.12234964508588764,
 'avg_cloudcover_low': -0.1223496450

In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Fit OLS model
X_with_const = sm.add_constant(X_scaled)  # Adds constant term to predictors
ols_model = sm.OLS(y, X_with_const).fit()

# Residuals
residuals = ols_model.resid

# Fitted values
fitted = ols_model.fittedvalues

# Residual vs Fitted plot for Linearity and Homoscedasticity
plt.scatter(fitted, residuals)
plt.axhline(0, color='r', linestyle='--')
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.show()

# QQ plot for Normality
sm.qqplot(residuals, line='s')
plt.show()

# VIF for Multicollinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
print(vif_data)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Assuming you have the dataframe df and reduced_columns list as in your screenshot

# Removing 'min_time' from the columns to plot
reduced_columns.remove('min_time')

# Setting up the figure and axes
fig, axes = plt.subplots(nrows=9, ncols=5, figsize=(20, 36))

# Flattening the axes array to loop through it
flat_axes = axes.flatten()

for ax, col in zip(flat_axes, reduced_columns):
    sns.histplot(df[col], ax=ax, kde=True, bins=30, color='blue')
    ax.set_title(col)
    ax.set_ylabel('Count')

# Remove the remaining unused subplots
for ax in flat_axes[len(reduced_columns):]:
    ax.remove()

# Adjust the layout to avoid overlap and display the plots
plt.tight_layout()
plt.show()


In [ ]:
import statsmodels.api as sm

def get_summary_statistics_multivariate(df, predictors, target="target_cost"):
    X = df[predictors]
    X = sm.add_constant(X)
    y = df[target]

    model = sm.OLS(y, X).fit()
    return model # model.summary()

# Assuming reduced_df is already defined
predictors = [col for col in reduced_df.columns if col != "target_cost"]
model = get_summary_statistics_multivariate(reduced_df, predictors)

print(model.summary())


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_coefficients(model, predictors):
    coef = model.params.drop('const').sort_values(ascending=False)
    std_err = model.bse[coef.index]
    
    plt.figure(figsize=(20, 15))
    coef.plot(kind='barh', xerr=std_err, grid=True, color='red')
    plt.yticks(np.arange(len(predictors)), coef.index)
    plt.xlabel('Coefficient Value')
    plt.title('OLS Coefficients with Standard Error Bars')
    plt.tight_layout()
    plt.show()

# Assuming the model is already fitted using the previous code
model = sm.OLS(reduced_df['target_cost'], sm.add_constant(reduced_df[predictors])).fit()
plot_coefficients(model, predictors)
